### 네이버 쇼핑 키워드 인사이트

#### 개인과제
- 키워드에 대한 검색량 데이터 수집
- 검색어에 대한 상품등록수 수집
- 수집한 데이터에 대한 인사이트 도출 : 예시) 경쟁률

#### 조별과제
- 스마트 스토어를 운영한다고 생각하고 경쟁력이 낮으면서 많은 수익을 낼수 있는 키워드와 상품 찾기
    - 예시외의 추가적인 페이지 데이터 수집하여 프로젝트 수행가능

#### 참고 서비스
- 신사임당 유투브 : https://youtu.be/Z0hL9khbpIw
- 응용 서비스 : https://whereispost.com/keyword, https://itemscout.io

#### 데이터 수집 프로세스
1. 네이버 검색광고 회원가입 및 로그인 : https://searchad.naver.com
2. 광고 시스템 > 도구 > API 사용관리
3. CUSTOMER_ID, API_KEY, SECRET_KEY 획득
4. 네이버 검색 광고 API 매뉴얼 사이트 참고 : http://naver.github.io/searchad-apidoc/#/guides
5. github 페이지 참고
    - https://github.com/naver/searchad-apidoc
    - https://github.com/naver/searchad-apidoc/blob/master/python-sample/examples/ad_management_sample.py
6. 키워드 입력하면 해당 키워드에 대한 데이터 수집

- hint
    - github 페이지의 코드에서 signaturehelper.py 파일을 이용하여 header를 만들어야 합니다.

In [1]:
import requests, json, time
import pandas as pd
import signaturehelper
import warnings
warnings.filterwarnings('ignore')

#### 1. 개별과제 : 키워드에 대한 검색량 데이터 수집

In [2]:
CUSTOMER_ID = 2481968
API_KEY = "01000000008855baccfe72e30f108e4eda21ca351143582e022fa357cb35b6e0cf98223752"
SECRET_KEY = "AQAAAACIVbrM/nLjDxCOTtohyjURH6P6HiAWC9KrjxfmOVrNAw=="

In [3]:
def get_header(method, uri, api_key, secret_key, customer_id):
    timestamp = str(round(time.time() * 1000))
    signature = signaturehelper.Signature.generate(timestamp, method, uri, SECRET_KEY)
    return {'Content-Type': 'application/json; charset=UTF-8', 'X-Timestamp': timestamp, 
            'X-API-KEY': API_KEY, 'X-Customer': str(CUSTOMER_ID), 'X-Signature': signature}

In [4]:
def keyword_master(keyword):
    url = f"https://api.naver.com/keywordstool?hintKeywords={keyword}&showDetail=1"
    headers = get_header("GET", '/keywordstool', API_KEY, SECRET_KEY, CUSTOMER_ID)
    response = requests.get(url, headers=headers)
    return pd.DataFrame(response.json()["keywordList"])

In [5]:
keyword = "실내자전거"
df = keyword_master(keyword)

In [6]:
df.head()

,relKeyword,monthlyPcQcCnt,monthlyMobileQcCnt,monthlyAvePcClkCnt,monthlyAveMobileClkCnt,monthlyAvePcCtr,monthlyAveMobileCtr,plAvgDepth,compIdx
0,실내자전거,19400,163400,112.1,2643.0,0.66,1.76,15,높음
1,하체운동기구,1070,7790,8.0,120.8,0.81,1.66,15,높음
2,헬스자전거,1240,11900,17.4,318.0,1.56,2.86,15,높음
3,스피닝자전거,2330,24200,12.5,360.5,0.58,1.56,15,높음
4,스쿼트머신,2260,16800,5.0,195.0,0.24,1.23,15,높음


#### 2. 개별과제 : 검색어에 대한 상품등록수 수집
- item_count : 등록된 상품수
- total_search_count : PC, Mobile 을 합친 검색량

In [7]:
from scrapy.http import TextResponse

In [10]:
def item_count(keyword):
    url = f"https://search.shopping.naver.com/search/all?query={keyword}"
    request = requests.get(url)
    response = TextResponse(request.url, body=request.text, encoding="utf-8")
    try:
        return response.xpath('//*[@id="__next"]/div/div[2]/div[2]/div[3]/div[1]/div[1]/ul/li[1]/a/span[1]/text()')[0].extract()
    except:
        return response.xpath('//*[@id="__next"]/div/div[2]/div/div[3]/div[1]/div[1]/ul/li[1]/a/span[1]/text()')[0].extract()

In [12]:
item_dfs = df.copy()

In [13]:
item_dfs["item_count"] = ""

In [14]:
for idx, data in item_dfs[:20].iterrows():
    print(idx, data["relKeyword"], item_count(data["relKeyword"]), end=" ")
    item_dfs.loc[idx, "item_count"] = item_count(data["relKeyword"])

0 실내자전거 255,224 1 하체운동기구 57,580 2 헬스자전거 248,789 3 스피닝자전거 63,138 4 스쿼트머신 46,537 5 다리운동기구 74,738 6 실내운동기구 348,247 7 실내사이클 144,817 8 실내싸이클 144,817 9 스핀바이크 23,136 10 라이딩 1,240,106 11 실내자전거렌탈 597 12 자전거운동기구 162,686 13 홈트운동기구 294,302 14 헬스운동기구 1,934,263 15 헬스용자전거 5,727 16 실내용자전거 14,918 17 폼롤러 332,358 18 운동기구렌탈 12,356 19 허벅지운동기구 43,089 

In [15]:
columns = ["relKeyword", "monthlyPcQcCnt", "monthlyMobileQcCnt", "item_count"]
result_df = item_dfs[columns][:20]
result_df

,relKeyword,monthlyPcQcCnt,monthlyMobileQcCnt,item_count
0,실내자전거,19400,163400,"255,224"
1,하체운동기구,1070,7790,"57,580"
2,헬스자전거,1240,11900,"248,789"
3,스피닝자전거,2330,24200,"63,140"
4,스쿼트머신,2260,16800,"46,537"
5,다리운동기구,350,4290,"74,738"
6,실내운동기구,1520,13400,"348,247"
7,실내사이클,3630,32600,"144,816"
8,실내싸이클,1250,11700,"144,817"
9,스핀바이크,2170,10400,"23,136"


#### 3. 개별과제 : 경쟁률
- 아이템수/검색수 : 높을수록 경쟁이 심한 상품 키워드
- 수집된 데이터에서 추가로 얻을수 있는 인사이트 추가 : 1개이상
    - 논리적인 인사이트이면 인사이트 수준에 대한 평가는 하지 않음

In [16]:
result_df["item_count"] = result_df["item_count"].apply(lambda data: data.replace(",", ""))
result_df["total_search_count"] = result_df["monthlyPcQcCnt"] + result_df["monthlyMobileQcCnt"]
result_df.head(2)

,relKeyword,monthlyPcQcCnt,monthlyMobileQcCnt,item_count,total_search_count
0,실내자전거,19400,163400,255224,182800
1,하체운동기구,1070,7790,57580,8860


In [19]:
for column in result_df.columns[1:]:
    result_df[column] = result_df[column].astype("int")
result_df.dtypes

relKeyword            object
monthlyPcQcCnt         int32
monthlyMobileQcCnt     int32
item_count             int32
total_search_count     int32
dtype: object

In [20]:
result_df["total_search_count"] = result_df["monthlyPcQcCnt"] + result_df["monthlyMobileQcCnt"]
result_df.head(2)

,relKeyword,monthlyPcQcCnt,monthlyMobileQcCnt,item_count,total_search_count
0,실내자전거,19400,163400,255224,182800
1,하체운동기구,1070,7790,57580,8860


In [23]:
# 실내자전거렌탈의 키워드가 경쟁령이 낮다.
result_df["competition_rate"] = result_df["item_count"] / result_df["total_search_count"]
result_df = result_df.sort_values("competition_rate").reset_index(drop=True)
result_df

,relKeyword,monthlyPcQcCnt,monthlyMobileQcCnt,item_count,total_search_count,competition_rate
0,실내자전거렌탈,90,750,597,840,0.710714
1,실내자전거,19400,163400,255224,182800,1.396193
2,스핀바이크,2170,10400,23136,12570,1.840573
3,스피닝자전거,2330,24200,63140,26530,2.379947
4,스쿼트머신,2260,16800,46537,19060,2.441605
5,폼롤러,14900,115300,332358,130200,2.552673
6,실내사이클,3630,32600,144816,36230,3.997129
7,실내용자전거,370,2480,14918,2850,5.234386
8,하체운동기구,1070,7790,57580,8860,6.498871
9,허벅지운동기구,510,4380,43089,4890,8.811656


#### 4. 조별과제 : 스마트 스토어를 운영한다고 생각하고 경쟁력이 낮으면서 많은 수익을 낼수 있는 키워드 찾기
- 정답이 있는것은 아닙니다
- 여러가지 가정과 데이터를 수집, 분석하여 판매할 상품과 키워드를 선정해 보시면 됩니다.
- 앞서 2일간 했던 프로젝트와 같은 방법으로 수행하시면 됩니다.

### 평가기준

#### 개인평가(70)

    - 기본 문제 해결(30)
        - 데이터 수집 방법 : 10점
            - 최적의 수집 방법을 사용했는지 확인
        - 수집된 데이터의 결과 : 20점
            - 1번: 키워드에 대한 검색량 데이터 수집 : 15점
            - 2번: 검색어에 대한 상품등록수 수집 : 5점
            
    - 적절한 주석 및 설명(10)
        - 코드의 가독성 및 코드의 설명 : 10점
        
    - 코드 오류(10)
        - 발생 오류마다 : -2점
        
    - 추가 분석(20)
        - 인사이트 도출
            - 경쟁률 데이터 출력 : 10점
            - 추가 인사이트 : 10점
            
#### 조별평가(30)
    
    - 비지니스 인사이트 도출(5)
        - 스마트 스토어 운영자 입장에서의 추가 인사이트 도출
        - 예시) 선정한 상품으로 소호몰에서 1위 하였을경우 예상되는 매출
        - 인사이트 도출을 위한 프로세스
        
    - 실습문제의 분석(5)
        - 제출 자료가 잘 정리되고 판매 상품과 키워드를 논리적으로 잘 선정
        - 상품과 키워드 선정 자동화 > 가독성 좋은 키워드와 상품 선정 프로세스
        
    - 실습문제의 적절한 정리(20)
        - 프로젝트 주제(상품과 키워드 선정)에 맞는 논리적 인사이트가 있는지 확인
        - 프로젝트 수행 후 추후 과제 및 느낀점